In [1]:
import sys
import os
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain

from langchain_core.output_parsers import JsonOutputParser

import plotly 
import plotly.graph_objs as go


Use cases: 

Communication with the Court
1. Casenet: 
- Receipts (Confirmation of Filing)
- Returned (Rejected) Notice
- E-Notice (another party files something on the case on Casenet)

2. TylerTech : IL Cook County
- Only one mail sent 

3. Direct Email from the Court

4. Direct Email to the Court with a response 

Communication with the Prosecutor 
- Acceptance 
- Rejection 
- Request for more information

Communication with the Client
- Initial marketing email


Next Steps: 
- Card should have the following information 
    - Case Number
    - Court Date / Court Time 
    - Court/Court Location
    - Charges 
- Rank the cases by next court date / court time 
- Refresh the open cases every day from Casenet to update the court date / time 
    - A case is closed if we have anything expect disposed 
- Analyze all the emails and update the status and mark them as new when new update based on detemerministic rules
- Level#2: Analyze the text inside the emails 
- Level#3: Analyze the last paid invoices 
- Suggestion from the beginning should contain the appropriate action 

In [2]:
OPENAI_API_VERSION = "2024-02-15-preview"
AZURE_OPENAI_KEY = "d7878ff58933461a8fd46e35e018c714"
AZURE_OPENAI_ENDPOINT = "https://fublo.openai.azure.com/"
AZURE_OPENAI_CHATGPT_DEPLOYMENT = "gpt-35-turbo-16k"

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION
os.environ["AZURE_OPENAI_CHATGPT_DEPLOYMENT"] = AZURE_OPENAI_CHATGPT_DEPLOYMENT

In [3]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
)

In [4]:
sys.path.append("..")
os.environ["ROOT_PATH"] = "/Users/aennassiri/Projects/Personal/ticket-washer"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/Users/aennassiri/Projects/Personal/ticket-washer/configuration/fubloo-app-1f213ca274de.json"
)

In [5]:
from src.services.emails import GmailConnector


In [6]:
user_id = "ayoub@tickettakedown.com"

In [7]:
gmail_connector = GmailConnector(user_id=user_id)


In [8]:
emails = gmail_connector.get_inbox_emails(total_results=20000)

Got the first 20 emails
Got the first 40 emails
Got the first 60 emails
Got the first 80 emails
Got the first 100 emails
Got the first 120 emails
Got the first 140 emails
Got the first 160 emails
Got the first 180 emails
Got the first 200 emails
Got the first 220 emails
Got the first 240 emails
Got the first 260 emails
Got the first 280 emails
Got the first 300 emails
Got the first 320 emails
Got the first 340 emails
Got the first 360 emails
Got the first 380 emails
Got the first 400 emails
Got the first 420 emails
Got the first 440 emails
Got the first 460 emails
Got the first 480 emails
Got the first 500 emails
Got the first 520 emails
Got the first 540 emails
Got the first 560 emails
Got the first 580 emails
Got the first 600 emails
Got the first 620 emails
Got the first 640 emails
Got the first 660 emails
Got the first 680 emails
Got the first 700 emails
Got the first 720 emails
Got the first 740 emails
Got the first 760 emails
Got the first 780 emails
Got the first 800 emails
Got 

In [10]:
len(emails)

email = emails[34]

In [11]:
print(f"Found total of {len(emails)} emails")

Found total of 20000 emails


In [12]:
email

{'id': '18ebf8c5411dbe4c',
 'snippet': 'Thank you for using the Missouri Judiciary Pay By Web system to make a payment. Below is a copy of the information pertaining to this transaction. Payments typically will be receipted by the court',
 'payload': {'partId': '',
  'mimeType': 'multipart/mixed',
  'filename': '',
  'headers': [{'name': 'Delivered-To',
    'value': 'meyerattorneyservices@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a17:907:54f:b0:a51:aaa7:9f75 with SMTP id wk15csp1903075ejb;        Mon, 8 Apr 2024 14:10:29 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AGHT+IHf3jGTJAceyrkWbHWnwuQhj1nUKKICW8RHWiEJXo3qzmQV8HQGWHKxsYUVj0V7jLMJLd4F'},
   {'name': 'X-Received',
    'value': 'by 2002:a6b:770d:0:b0:7d5:c016:3ae6 with SMTP id n13-20020a6b770d000000b007d5c0163ae6mr10395598iom.2.1712610628687;        Mon, 08 Apr 2024 14:10:28 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1712610628; cv=none;        d=google.com; s=arc-

In [13]:
gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")

'2024-04-08'

In [14]:
 # Subject
gmail_connector.get_snippet(email)


'Thank you for using the Missouri Judiciary Pay By Web system to make a payment. Below is a copy of the information pertaining to this transaction. Payments typically will be receipted by the court'

In [15]:
gmail_connector.get_email_subject(email)   # Subject

'Pay By Web Payment Confirmation - PBW851343 - 211320723 - CITY OF ST. LOUIS V EDWIN VONDER SHUMAN, St Louis City Municipal Court'

In [16]:
      # Sender
gmail_connector.render_email_sender(gmail_connector.get_sender(email))

'Missouri Courts Pay By Web System'

In [85]:
import re

def extract_number_from_string(s):
    output = None
    match = re.search(r'\b\d+\b', s)
    if match:
        output = match.group(0)
    # Format 2316-CR04348
    match = re.search(r'\b\w{4}-\w{2}\w{5}\b', s)
    if match:
        output = match.group(0)

    if output is not None and len(output) < 5:
        output = None
    return output

In [87]:
import re

# The text strings to search
texts = [
    "Receipt of Submission - 704258650 - ST V GARRETT JACOB GARING, Randolph County - Circuit Court - Huntsville",
    "eNotice - 210900213 - KANSAS CITY V JACKSON FLYNN CONROE, Kansas City Municipal Court",
    "eNotice - 2416-CV09157 - TERENCE TENSLEY V DIRECTOR OF REVENUE, Jackson County - Independence - Civil",
    "eNotice - 2316-CR04348 - ST V SEAN F MOLBY, Jackson County - Independence - Criminal/Traffic",
    "eNotice - 24CT-CR00287-01 - ST V AMANDA MICHELLE RIDDLE, Christian County - Circuit Court",
    "77% OFF is still available!",
]

# Regular expression to extract numbers
pattern = r'\b\d+\b'

# Extracting numbers
extracted_numbers = [extract_number_from_string(text) for text in texts]
extracted_numbers

['704258650',
 '210900213',
 '2416-CV09157',
 '2316-CR04348',
 '24CT-CR00287',
 None]

In [88]:
email_template = f"""
Date : {gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")} \n

Subject: {gmail_connector.get_email_subject(email)} \n

Body: {gmail_connector.get_snippet(email)} \n

Sender: {gmail_connector.render_email_sender(gmail_connector.get_sender(email))} \n

"""
email_template

'\nDate : 2023-03-28 \n\n\nSubject: Re: 704006187 - ST V JENNIFER ELSIE SAWYER, Dunklin County - Circuit Court \n\n\nBody: I can definitely understand that. The lying part probably put it over the top. Is there any other resolution we can come to on this one? May SIS or DIP? Thank you for your time and I understand either \n\n\nSender: Shawn-Anthony Meyer \n\n\n'

In [22]:
gmail_connector.get_snippet(email)

'Thank you for using the Missouri Judiciary Pay By Web system to make a payment. Below is a copy of the information pertaining to this transaction. Payments typically will be receipted by the court'

In [23]:
extract_number_from_string(gmail_connector.get_email_subject(email))

'211320723'

In [132]:
from src.services.cases import get_single_case, patch_case


In [25]:
get_single_case("240065833")

Case(case_id='240065833', court_id='SMPDB0001_CT06', flag='pending', urgent=None, participants=['Sid5dOG03tMxIdQ6UlyK', 'GC7nwDYpbsyqFc0kdCgE'], related_cases=None, protection_order=False, parties=[{'other_attorneylist': [], 'pred_code': 'Z^', 'desc': 'Defendant', 'pty_seqno': 2, 'birth_date': '11/29/1976', 'lit_ind': 'Y', 'mid_initial': 'A', 'pidm': 90241667, 'addr_line1': '4722 N MULBERRY CT', 'addr_city': 'KANSAS CITY', 'birth_datecode': 'Y', 'desc_code': 'DFT', 'formatted_partyaddress': '4722 N MULBERRY CT\nKANSAS CITY, MO 64116', 'last_name': 'DICKEMANN', 'addr_statcode': 'MO', 'co_attorneylist': [], 'formatted_telephone': '', 'attorney_list': [], 'addr_atyp': 'MA', 'formatted_birthdate': '1976', 'prosecuting_atty': False, 'criminal_ind': 'Y', 'sort_seq': 4, 'addr_seqno': 2, 'assoc': 0, 'addr_zip': '64116', 'first_name': 'DEREK', 'party_type': 'party', 'formatted_partyname': 'DICKEMANN, DEREK A', 'criminal_case': True, 'start_date': '02/28/2024'}, {'addr_statcode': 'KS', 'other_at

In [26]:
case_statuses = {
    "filed": {
        "name": "Case Filed",
        "owner": "Client",
        "mandatory": True,
        "short_description": "Case Filed on Casenet",
        "internal_status": "Case Filed",
        "client_status": "",
        "next_case_status": "",
    },
    "paid": {
        "name": "Case Payment",
        "owner": "Client",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Client Paid",
        "internal_status": "Payment Received",
        "client_status": "",
        "next_case_status": "REVIEW",
        "color": "pending",
    },
    "review": {
        "name": "Case Review and Consultation",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Case Under Review",
        "internal_status": "Under Review",
        "client_status": "",
        "next_case_status": "REJECTED, EOA",
        "color": "orange",
    },
    "rejected": {
        "name": "Case Rejected",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Case Rejected",
        "internal_status": "Case cannot be handled by our attorneys",
        "client_status": "",
        "next_case_status": "REFUND",
        "color": "red",
    },
    "refund": {
        "name": "Case Refund",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "closed",
        "short_description": "Case Refunded",
        "internal_status": "We initiated a full refund. It should be reflected in your account within 5-10 business days. We apologize for any inconvenience.",
        "client_status": "",
        "next_case_status": "CLOSURE",
        "color": "red",
    },
    "sign_engagement_letter": {
        "name": "Sign Engagement Letter",
        "owner": "Client, Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Engagement Letter Sign-Off",
        "internal_status": "Case Accepted",
        "client_status": "Our Attorneys have accepted your case",
        "next_case_status": "EOA",
        "color": "orange",
    },
    "warrant_status": {
        "name": "Warrant Status",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Recall Warrant",
        "internal_status": "Warrant",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "eoa": {
        "name": "Entry of Appearance",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "EOA",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT, CLIENT_REQUEST",
        "color": "green",
    },
    "court_response": {
        "name": "Court Response",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Court Response",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "case_update": {
        "name": "Case Update",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Case Update",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "court_returned_for_filing": {
        "name": "Court Returned for Filing",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Court Returned for Filing",
        "internal_status": "Our Attorneys are Working on the Case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "client_request": {
        "name": "Documentation Request from the Customer",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Client Intake Form",
        "internal_status": "Our Attorneys have requested more information from you on your case",
        "client_status": "",
        "next_case_status": "REV_INT",
        "color": "orange",
    },
    "rev_int": {
        "name": "Case Documentation & Information Gathering",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Internal Review",
        "internal_status": "Our Attorneys are gathering documentation to back your case",
        "client_status": "",
        "next_case_status": "DEFENSE_REV",
        "color": "orange",
    },
    "defense_dev": {
        "name": "Case Legal Strategy Development",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Review with the Client",
        "internal_status": "Our attorneys are elaborating a strategy to defend your cause",
        "client_status": "",
        "next_case_status": "REC_REQUEST",
        "color": "orange",
    },
    "rec_request": {
        "name": "Case Request for Recommendation",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "RFR Filing",
        "internal_status": "Attorney has initiated negotiations with the Prosecutor",
        "client_status": "",
        "next_case_status": "REC_RECEIVED, REC_DELAYED, REC_REJECT",
    },
    "rec_received": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Received",
        "internal_status": "Attorney has negotiated a favorable plea offer",
        "client_status": "",
        "next_case_status": "REC_REVIEW",
    },
    "rec_reject": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Rejected",
        "internal_status": "Case cannot be handled by our attorneys",
        "client_status": "",
        "next_case_status": "REFUND",
    },
    "rec_delay": {
        "name": "Case Request for Recommendation",
        "owner": "Prosecutor",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Delayed",
        "internal_status": "The Prosecutor has delayed providing a plea offer. Please provide the information requested",
        "client_status": "",
        "next_case_status": "CLIENT_REQUEST, REC_RECEIVED, REC_REVIEW",
    },
    "rec_review": {
        "name": "Case Plea Negotiation with the Prosecutor (if applicable)",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "todo",
        "short_description": "Recommendation Review",
        "internal_status": "The Attorney has initiated further negotiations with the prosecutor",
        "client_status": "",
        "next_case_status": "REC_REVIEW, REC_PROPOSAL",
    },
    "rec_proposal": {
        "name": "Case Recommendation Proposal",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation Proposed to Client",
        "internal_status": "Attorney has negotiated a favorable plea offer and would like you to review and accept the plea offer",
        "client_status": "",
        "next_case_status": "SIG_REQUIRED",
    },
    "sig_required": {
        "name": "Case pending Signature and Payment",
        "owner": "Client",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation pending Signature",
        "internal_status": "Client signature and payment are required to accept the Prosecutor's Plea Offer",
        "client_status": "",
        "next_case_status": "SUB",
    },
    "sub": {
        "name": "Case Recommendation and Payment Submission to Court",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "todo",
        "short_description": "Recommendation to Submit to Court",
        "internal_status": "Attorney is submitting the signed plea and payment to the court for approval",
        "client_status": "",
        "next_case_status": "SUB_REVIEW",
    },
    "sub_review": {
        "name": "Case Recommendation and Payment Review by the Court",
        "owner": "Court",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Recommendation Submission under Review by the Court",
        "internal_status": "Plea Offer is pending Court Approval",
        "client_status": "",
        "next_case_status": "APPEAR, CLOSE",
    },
    "appear": {
        "name": "Court Representation (if applicable)",
        "owner": "Lawyer",
        "mandatory": False,
        "flag": "pending",
        "short_description": "Court Appearance Required",
        "internal_status": "Court is requiring an appearance to approve the plea",
        "client_status": "",
        "next_case_status": "CLOSE",
    },
    "close": {
        "name": "Case Closure",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "pending",
        "short_description": "Close Case on Portal",
        "internal_status": "Congratulations! Case closed",
        "client_status": "",
        "next_case_status": "CLIENT_REVIEW",
    },
    "client_review": {
        "name": "Case Feedback",
        "owner": "Client",
        "mandatory": False,
        "flag": "closed",
        "short_description": "Client Submits Review",
        "internal_status": "Please provide us with a review and feedback from your experience",
        "client_status": "",
        "next_case_status": "CLIENT_FEEDBACK",
    },
    "closed": {
        "name": "Case Closed",
        "owner": "Lawyer",
        "mandatory": True,
        "flag": "closed",
        "short_description": "Case Closed",
        "internal_status": "Case closed",
        "client_status": "",
        "next_case_status": "",
    },
}

In [59]:
"""
Information received from the court	2113	=> Update the case	=> Update the case status to court_response
Reply on a request for recommendation	286	=> Update the case	=> Update the case status to rec_received
Returned for Filing 					168	=> Update the case 	=> Update the case status to court_returned_for_filing
Update the case 						565 => Update the case	=> Update the case status to case_update
"""

rules = [
    {
        # Sender shouldnt be meyerattorneyservices@gmail.com
        "regex": r"meyerattorneyservices",
        "case_status": None,
        "scope": "sender",
    },
    {
        # Sender shouldnt be Shawn-Anthony Meyer
        "regex": r"shawn-anthony meyer",
        "case_status": None,
        "scope": "sender",
    },
    {
        "regex": r"returned for filing",
        "case_status": "court_returned_for_filing",
        "scope": "subject",
    },
    {
        "regex": r"request for recommendation",
        "case_status": "rec_received",
        "scope": "subject",
    },
    {
        "regex": r"enotice",
        "case_status": "court_response",
        "scope": "subject",
    },
]

In [89]:
def get_action_from_rules(email_dict, rules):
    action = None
    for rule in rules:
        if rule["scope"] == "subject":
            match = re.search(rule["regex"], email_dict["subject"], re.IGNORECASE)
            if match:
                action = rule["case_status"]
                break
        elif rule["scope"] == "body":
            match = re.search(rule["regex"], email_dict["body"], re.IGNORECASE)
            if match:
                action = rule["case_status"]
                break
        elif rule["scope"] == "sender":
            match = re.search(rule["regex"], email_dict["sender"], re.IGNORECASE)
            if match:
                break
    return action

In [107]:
emails_list = []

for email in emails:
    case_number = extract_number_from_string(gmail_connector.get_email_subject(email))
    subject = gmail_connector.get_email_subject(email)
    if case_number is None:
        case_number = extract_number_from_string(gmail_connector.get_snippet(email))
    sender = gmail_connector.render_email_sender(gmail_connector.get_sender(email))
    timestamp = gmail_connector.get_timestamp(email).strftime("%Y-%m-%d")
    email_dict = {
        "case_number": case_number,
        "subject": subject,
        "sender": sender,
        "timestamp": timestamp,
    }

    action = get_action_from_rules(email_dict, rules)
    
    if action:
        email_dict["action"] = action

    emails_list += [email_dict]

In [108]:
import pandas as pd 
# Change the backend for pandas to plotly
pd.options.plotting.backend = "plotly"

emails_df = pd.DataFrame(emails_list)
emails_df["timestamp"] = pd.to_datetime(emails_df["timestamp"])

In [109]:
# Plot the number of emails received per day
emails_df["timestamp"].value_counts().sort_index().plot()

/Users/aennassiri/Projects/Personal/ticket-washer/venv/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [110]:
# Number of emails with actions 
emails_df["action"].value_counts().plot(kind="bar")

In [111]:
# Focus on 2024 
emails_df = emails_df[emails_df["timestamp"].dt.year == 2024]

In [112]:
# Sort the emails by timestamp
emails_df = emails_df.sort_values(by="timestamp", ascending=False)

In [119]:
# Count cases with many emails 
emails_df.groupby("case_number").timestamp.count().sort_values(ascending=False)

case_number
72934           81
23JE-CR03544    39
24SA-CR00019    37
704283738       32
23CW-CR00594    31
                ..
220210358        1
210466458        1
210258815        1
210258812        1
035361           1
Name: timestamp, Length: 958, dtype: int64

In [123]:
# Group the emails by case number and keep the last email
cases_focus = emails_df.groupby("case_number").first().reset_index()

In [124]:
# Focus on 23JE-CR03544
emails_df[emails_df["case_number"] == "23JE-CR03544"]


,case_number,subject,sender,timestamp,action
11,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-09,court_response
451,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-03,court_response
455,23JE-CR03544,RE: Motion to Recall Warrant - 23JE-CR03544 - ...,Kara Riffe,2024-04-03,NaN
522,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-03,court_response
493,23JE-CR03544,Returned for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,court_returned_for_filing
497,23JE-CR03544,Receipt of Submission - 23JE-CR03544 - ST V MI...,Missouri Courts eFiling System,2024-04-03,NaN
498,23JE-CR03544,Motion to Recall Warrant - 23JE-CR03544 - ST V...,Shawn-Anthony Meyer,2024-04-03,NaN
492,23JE-CR03544,Returned for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,court_returned_for_filing
501,23JE-CR03544,Accepted for Filing - 23JE-CR03544 - ST V MICH...,Missouri Courts eFiling System,2024-04-03,NaN
502,23JE-CR03544,Receipt of Submission - 23JE-CR03544 - ST V MI...,Missouri Courts eFiling System,2024-04-03,NaN


In [125]:
cases_focus[cases_focus["case_number"] == "23JE-CR03544"]

,case_number,subject,sender,timestamp,action
614,23JE-CR03544,eNotice - 23JE-CR03544 - ST V MICHAEL WAYNE GR...,Missouri Courts eFiling System,2024-04-09,court_response


In [99]:
emails_df.to_csv("emails.csv", index=False)

In [126]:
cases_focus.to_csv("cases.csv", index=False)

In [131]:
# Iterate through case_focus and get the case status
for index, row in cases_focus.iterrows():
    case_number = row["case_number"]
    case = get_single_case(case_number)
    if case is None:
        print(f"Case {case_number} not found in the DB")
        print(row)
    else:
        # Update the case status
        case_status = row["action"]
        if case.status != case_status and case.status != "closed":
            print(f"Updating case {case_number} from {case.status} to {case_status}")
            patch_case(case_number, {"status": case_status})

Case 035361 not found in the DB
case_number                    035361
subject          Your single-use code
sender         Microsoft account team
timestamp         2024-03-29 00:00:00
action                           None
Name: 0, dtype: object
Case 068836 not found in the DB
case_number                              068836
subject        Microsoft account password reset
sender                   Microsoft account team
timestamp                   2024-03-04 00:00:00
action                                     None
Name: 1, dtype: object
Case 10016939490803 not found in the DB
case_number                                   10016939490803
subject        Your monthly account statement is here, Shawn
sender                                              Experian
timestamp                                2024-04-08 00:00:00
action                                                  None
Name: 2, dtype: object
Case 117766 not found in the DB
case_number                                               11

NameError: name 'patch_case' is not defined

In [130]:
case.case_status

In [61]:
emails_by_sender = emails_df.groupby("sender").id.count().sort_values(ascending=False)

In [63]:
emails_by_sender.to_csv("emails_by_sender.csv")

In [22]:
import json
import os

from langchain.callbacks import get_openai_callback
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate

In [112]:
def get_email_info_structured(text: str) -> str:
    """
    This function would return the result in JSON structure only.
    We don't have to write a custom parse to convert string to JSON LangChain already provides with the a parser
    """
    response_schemas = [
        ResponseSchema(name="Summary", description="Summary of the email"),
        ResponseSchema(
            name="CaseNumber",
            description="Case Number extracted from the email",
        ),
        # Sentiment Analysis
        ResponseSchema(
            name="Sentiment",
            description="Sentiment of the email",
        ),
        # Next Steps 
        ResponseSchema(
            name="NextCaseStatus",
            description="Next Case Status",
        ),
        # Next Step Owner 
        ResponseSchema(
            name="NextStepOwner",
            description="Next Step Owner",
        )
    ]
    output_parser = StructuredOutputParser.from_response_schemas(
        response_schemas
    )

    format_instructions = output_parser.get_format_instructions()
    prompt = PromptTemplate(
        template=f"""Get the summary of the email and extract the case number from the email and provide the sentiment of the email and the next case status and the next step owner. The current process is as follows: {case_statuses}""",
        input_variables=["text"],
        partial_variables={"format_instructions": format_instructions},
    )

    chain = prompt | llm | output_parser
    with get_openai_callback() as cb:
        response = chain.invoke({"filed": text})
        print(f"Total Cost (USD): ${format(cb.total_cost, '.4f')}")

    return response

In [113]:
final_body, text_body = gmail_connector.get_email_html_body(email)

In [116]:
text_body

prompt = f"""Get the summary of the email and extract the case number from the email and provide the sentiment of the email and the next case status and the next step owner. #### The email :{text_body} #### The current process is as follows: {case_statuses}"""

In [108]:
final_body, text_body = gmail_connector.get_email_html_body(email)

In [109]:
text_body

"      \r I can amend to Defective Equipment, $200 total. I'll send you a copy of the amended ticket for filing with payment. Joe Cambiano   From: meyerattorneyservices@gmail.com <meyerattorneyservices@gmail.com> Sent: Tuesday, April 2, 2024 12:45 PM To: Joe Cambiano <joe@kcinjurylaw.com> Subject: Request for Recommendation - BRYCEN DON THORNTON - 190873126   Hello, \r I hope this email finds you well! \r I am reaching out on behalf of my client, BRYCEN DON THORNTON, who has sought my representation in a criminal matter presently overseen by your office. The details pertaining to the case are as follows: \r Case #: 190873126\r Charge: Fail To Stop At Stop Sign At Stop Line/Before Crosswalk/Point Nearest Intersection \r I am reaching out to request your recommendation regarding this matter. I have attached a copy of the citation(s) for your reference. Should you require any further information or documentation to facilitate your recommendation, please do not hesitate to let\r  me know. 

In [117]:
########################
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [121]:
system_message = f"You are an an attorney that work on traffic tickets. We received a response from the prosecutor on a case. I received from the prosecutor {gmail_connector.render_email_sender(gmail_connector.get_sender(email))} regarding a traffic ticket"

system_message_prompt = SystemMessagePromptTemplate.from_template(
    system_message
)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

chain = LLMChain(llm=llm, prompt=chat_prompt)
result = chain.run(prompt)
print(result)

Summary of the email:

The prosecutor, Joe Cambiano, responded to an attorney's request for a recommendation on behalf of their client, Brycen Don Thornton. The case number is 190873126, and the charge is "Fail To Stop At Stop Sign At Stop Line/Before Crosswalk/Point Nearest Intersection." The prosecutor suggested amending the charge to Defective Equipment with a total fine of $200. They will send a copy of the amended ticket for filing along with the payment.

Case Number: 190873126

Sentiment of the email: The email appears to be a response providing a plea offer, so the sentiment is likely neutral or slightly positive.

Next Case Status: The next case status is not mentioned in the email.

Next Step Owner: The next step owner is not mentioned in the email.
